In [1]:
import json
from pprint import pprint
import requests
import pickle
import sys
import os.path
import spacy
# from spacy_lookup import Entity
import numpy as np
from numpy import genfromtxt
import logging

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


# Environment

In [2]:
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(asctime)s %(name)-12s %(levelname)-8s %(message)s'))
logger = logging.getLogger(__name__)
logger.addHandler(handler)
logger.propagate = False
logger.setLevel(logging.INFO)

# Common Routines

# Real Code

In [4]:
if sys.platform == 'win32':
    dropbox_intermediate_path = 'C:\\Users\\dwane\\Dropbox\\Datasets\\intermediate\\'
else:
    dropbox_intermediate_path = '/Users/dsluis/Dropbox/Datasets/intermediate/'

my_data = genfromtxt(dropbox_intermediate_path+'dexter_light_features_v0001.txt', delimiter=',')



In [6]:
my_data

array([[0.00000000e+00, 4.10554562e-01, 4.10554562e-01, ...,
        0.00000000e+00, 1.11800000e+03,            nan],
       [0.00000000e+00, 7.67441860e-01, 7.67441860e-01, ...,
        0.00000000e+00, 1.11800000e+03,            nan],
       [0.00000000e+00, 7.42397138e-02, 7.42397138e-02, ...,
        0.00000000e+00, 1.11800000e+03,            nan],
       ...,
       [0.00000000e+00, 6.06666667e-01, 6.06666667e-01, ...,
        0.00000000e+00, 1.20000000e+03,            nan],
       [0.00000000e+00, 7.50000000e-01, 7.50000000e-01, ...,
        0.00000000e+00, 1.20000000e+03,            nan],
       [0.00000000e+00, 1.07500000e-01, 6.17500000e-01, ...,
        0.00000000e+00, 1.20000000e+03,            nan]])

In [10]:
y = my_data[:,0]
X = my_data[:,1:-1]

logger.info('y')
logger.info(y)
logger.info('X')
logger.info(X)




2018-06-20 14:08:48,830 __main__     INFO     y
2018-06-20 14:08:48,833 __main__     INFO     [ 0.  0.  0. ...,  0.  0.  0.]
2018-06-20 14:08:48,837 __main__     INFO     X
2018-06-20 14:08:48,839 __main__     INFO     [[  4.10554562e-01   4.10554562e-01   4.10554562e-01 ...,   0.00000000e+00
    0.00000000e+00   1.11800000e+03]
 [  7.67441860e-01   7.67441860e-01   7.67441860e-01 ...,   0.00000000e+00
    0.00000000e+00   1.11800000e+03]
 [  7.42397138e-02   7.42397138e-02   7.42397138e-02 ...,   0.00000000e+00
    0.00000000e+00   1.11800000e+03]
 ..., 
 [  6.06666667e-01   6.06666667e-01   6.06666667e-01 ...,   0.00000000e+00
    0.00000000e+00   1.20000000e+03]
 [  7.50000000e-01   7.50000000e-01   7.50000000e-01 ...,   0.00000000e+00
    0.00000000e+00   1.20000000e+03]
 [  1.07500000e-01   6.17500000e-01   3.80555556e-01 ...,   0.00000000e+00
    0.00000000e+00   1.20000000e+03]]


In [11]:
# Overwrite NaNs with 0s
print('Are there NaNs?')
nan_locations = np.argwhere(np.isnan(X))

for loc in nan_locations:
    X[loc[0],loc[1]] = 0

print('Are there NaNs?')
nan_locations = np.argwhere(np.isnan(X))


Are there NaNs?
Are there NaNs?


Base on code from:
http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html


In [12]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

In [18]:
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision



c:\programs\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programs\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programs\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programs\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programs\anaconda3\lib\site-packages\sklearn\metrics\classifi

Best parameters set found on development set:

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.481 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.481 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.481 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.481 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.644 (+/-0.123) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.481 (+/-0.000) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.641 (+/-0.045) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.481 (+/-0.000) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.633 (+/-0.042) for {'C': 1, 'kernel': 'linear'}
0.633 (+/-0.043) for {'C': 10, 'kernel': 'linear'}
0.638 (+/-0.025) for {'C': 100, 'kernel': 'linear'}
0.638 (+/-0.027) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

            

In [20]:

output_filename = dropbox_intermediate_path+'binary_classification_model_v0001.pickle'
logger.info('About to write %s', output_filename)
with open(output_filename, 'wb') as handle:
    pickle.dump(clf, handle, protocol=pickle.HIGHEST_PROTOCOL)
logger.info('file written = %s', output_filename)


2018-06-20 08:13:39,333 __main__     INFO     About to write C:\Users\dwane\Dropbox\Datasets\intermediate\binary_classification_model_v0001.pickle
2018-06-20 08:13:39,333 __main__     INFO     file written = C:\Users\dwane\Dropbox\Datasets\intermediate\binary_classification_model_v0001.pickle


In [6]:

filename = dropbox_intermediate_path+'binary_classification_model_v0001.pickle'
logger.info('Loading %s', filename)
with open(filename, 'rb') as handle:
    xxx = pickle.load(handle)
logger.info('Loaded %s', filename)

2018-06-20 13:55:20,874 __main__     INFO     Loading /Users/dsluis/Dropbox/Datasets/intermediate/binary_classification_model_v0001.pickle
/Users/dsluis/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVC from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/dsluis/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
2018-06-20 13:55:20,882 __main__     INFO     Loaded /Users/dsluis/Dropbox/Datasets/intermediate/binary_classification_model_v0001.pickle


In [7]:
print(xxx)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_macro', verbose=0)


In [18]:

m = SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)


m.fit(X_train, y_train)
    
    

[LibSVM]

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True)

In [19]:
y_hat = m.predict(X_test)




In [20]:
sum(y_hat)


464.0

In [21]:
sum(y_test)

1249.0

In [22]:
sum(y_hat == y_test)

31689

In [23]:
sum(y_hat != y_test)

1495

In [25]:
sum(y_hat[y_test == 1] != y_test[y_test == 1])

1140

In [26]:
sum(y_hat[y_test == 0] != y_test[y_test == 0])

355

In [1]:
from sel.binary_classifier import BinaryClassifierTrainer


In [2]:
bc = BinaryClassifierTrainer()

In [3]:
bc.load_light_parameter_data_fmt_02()

bc.predict()

2018-06-27 11:34:32,377 sel.binary_classifier INFO     loading data from : /Users/dsluis/Dropbox/Datasets/intermediate/dexter_light_features_fmt_v02_partial.03.docs-1-195.txt
2018-06-27 11:34:32,484 sel.binary_classifier INFO     Are there NaNs?
2018-06-27 11:34:32,495 sel.binary_classifier INFO     Are there NaNs?
2018-06-27 11:34:32,499 sel.binary_classifier INFO     []
2018-06-27 11:34:32,500 sel.binary_classifier INFO     X Shape (3431, 23)


(array([[  4.14593698e-04,   5.60530680e-01,   3.18822554e-01, ...,
           1.70000000e+01,   1.70000000e+01,   2.41200000e+03],
        [  4.15110004e-04,   1.56081362e-01,   7.82482358e-02, ...,
           1.06000000e+02,   1.23000000e+02,   2.40900000e+03],
        [  4.36871997e-04,   4.36871997e-04,   4.36871997e-04, ...,
           4.00000000e+01,   4.00000000e+01,   2.28900000e+03],
        ..., 
        [  2.56613757e-01,   7.23544974e-01,   4.90079365e-01, ...,
           3.40000000e+01,   4.00000000e+01,   7.56000000e+02],
        [  8.18879509e-01,   8.18879509e-01,   8.18879509e-01, ...,
           1.40000000e+01,   1.60000000e+01,   1.30300000e+03],
        [  9.45747801e-01,   9.45747801e-01,   9.45747801e-01, ...,
           4.00000000e+00,   4.00000000e+00,   2.04600000e+03]]),
 array([False, False, False, ...,  True,  True,  True], dtype=bool))